In [ ]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import xlwings as xw
from datetime import date
start_time = time.time()
print('1/14.- Reading the Current Backlog Combined Data')
df1 = pd.read_excel("dataATS.xlsx",dtype={'PART': str})
#df1 = pd.read_excel("dataAIT.xlsx", header=2, dtype={'PART': str})
df1['SHIP_LOC'] = df1['SHIP_LOC'].str[:3] #delete possible dc with an -
df1['SHIP_TO'] = df1['SOLD_TO'] #Copy AE column to AF
print('2.- Filtering file for ATS')
df1 = df1[(df1['EVM_AIT2'] == 'ATS') | (df1['EVM_AIT2'] == 'AIT')| (df1['EVM_AIT2'] == 'LS') | (df1['EVM_AIT2'] == 'PARTS')] 
#subset print(df1.shape)
#df1 = df1[(df1['EVM_AIT2'] != '')] #subset print(df1.shape)
#df1 = df1[(df1['EVM_AIT2'] != 'Circular Economy')]#subset print(df1.shape)
df1['SSD'] = pd.to_datetime(df1['SSD'])
df1['MATL_TYPE']=np.where(df1['MATL_TYPE']== 'Included','Simple',df1['MATL_TYPE'])
#df1.MATL_TYPE.unique()

print('3.- Inserting columns')
df1 = df1.drop(['QOH'], axis = 1)
df1.rename(columns={'NETTABLE_QOH': 'QOH'},inplace=True)

df1.insert(2, 'Trilogy Or Merlin', 'MERLIN')
df1.insert(3, 'Material Avail Date > CRSD', '') #J column formula
df1.insert(5, 'SKU Short(QTYs On Hand) DBL & CRSD <=+4', '')
df1.insert(6, 'SO Short(QTYs On Hand) DBL & CRSD <=+4', '')
df1.insert(7, 'Missing CRSD Wk', '')
df1.insert(8, 'CRSD WK', '')
df1.insert(9, 'TPC SKU/DC/WK Key', '')
df1["SKU DC Key"]=df1["PART"]+df1["SHIP_LOC"]
df1.insert(13, 'Blended ASP $', '')
df1.insert(14, 'EXT Part ASP $', '')
df1.insert(19, 'Oracle Intransit(Trilogy IR/ISO)', '')
df1.insert(20, 'Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)', '')
df1.insert(21, 'ZCM ASN QTY Supply', '')
df1.insert(22, 'ZCM Commits WK1', '')
df1.insert(23, 'ZCM Commits WK2', '')
df1.insert(24, 'ZCM Commits WK3', '')
df1.insert(25, 'ZCM Commits WK4', '')
df1.insert(26, 'ZCM Commits WK5', '')
df1.insert(27, 'WK1 TPC Receipt QTY', '')
df1.insert(28, 'Valid TPC Supply', '')
df1.insert(29, 'TPC Commits WK1', '')
df1.insert(30, 'TPC Commits WK2', '')
df1.insert(31, 'TPC Commits WK3', '')
df1.insert(32, 'TPC Commits WK4', '')
#df1.insert(33, 'ZCM ASN/Commit Net WK1 supply', '')
df1.insert(34, 'ZCM ASN QTY and WK1 Net Supply', '')
df1.insert(35, 'ZCM Net Commits WK2', '')
df1.insert(36, 'ZCM Net Commits WK3', '')
df1.insert(37, 'Valid TPC Net Supply WK1', 0)
df1.insert(38, 'TPC Net Commits WK2', 0)
df1.insert(39, 'TPC Net Commits WK3', 0)
df1.insert(40, 'Missing Commits TPC/ZCM WK2', '')
df1.insert(41, 'Net Supply capture by CRSD SKU/DC/WK', '')
df1.insert(42, 'Missing ASN Supply Due to CRSD WK', '')
df1.insert(43, 'Missing Commit Supply Due to CRSD WK', '')
df1.insert(44, 'Merlin/Trilogy Missing Supply Total', '')
df1.insert(45, 'SKU/DC Total Supply(OH only)', '')
df1.insert(46, 'QTY Short to Sales Orders(CUM) Only INV', '')
df1.insert(47, 'QTY Short to Sales Orders(NON-CUM) Only INV', '')
df1.insert(48, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM) Only INV)', '')
df1.insert(49, 'SKU/DC Total Supply(On Hand+ALL ASN)', '')
df1.insert(50, 'QTY Short to Sales Orders(CUM) (On Hand+ALL ASN', '')
df1.insert(51, 'QTY Short to Sales Orders(NON-CUM) On Hand+ALL ASN', '')
df1.insert(52, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) On Hand+ALL ASN', '')
df1.insert(53, 'SKU/DC Total Supply ALL ASNs+Commits(No Missing Commits)', '')
df1.insert(54, 'QTY Short to Sales Orders(CUM) including Missing Commits+Commits+ALL ASNs', '')
df1.insert(55, 'QTY Short to Sales Orders(NON-CUM) w/ Commits+Missing Commits+ALL ASNs', '')
df1.insert(56, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) INV w/ Commits+Missing Commits+ALL ASNs', '')
df1.insert(57, 'SKU/DC Total Supply(OH+ALL ASNs+all 3wk Commtsh)', '')
df1.insert(58, 'QTY Short to Sales Orders(CUM) OH+ALL ASNs+all 3wk Commts', '')
df1.insert(59, 'QTY Short to Sales Orders(NON-CUM) OH+ALL ASNs+all 3wk Commts', '')
df1.insert(60, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) OH+ALL ASNs+all 3wk Commts', '')
#df1.insert(61, 'Config Printer SKU', '')
#df1.insert(62, 'Config Printer SKU/DC', '')
df1.insert(63, 'Config Printer SKU/DC/WK Key', '')
#df1.insert(64, 'Org Part INV-Part Simple Config Total QOH', '')
df1.insert(65, 'Config SKU/DC WO Supply Simple Config', 0)
#df1.insert(66, 'Config SKU/DC INV-Simple Config', '')
#df1.insert(67, 'QOH (Part Simple Config + BASE+ WO Supply)', '')

#create week COLUMN Z
today = date.today()
coming_sunday = today + datetime.timedelta(days=-today.weekday()-1, weeks=1) #this sunday sunday 1
coming_sunday = coming_sunday.strftime('%Y-%m-%d') #format it
coming_monday = today + datetime.timedelta(days=-today.weekday(), weeks=1)#this monday monday 1
coming_monday = coming_monday.strftime('%Y-%m-%d') #format it
next_sunday = today + datetime.timedelta(days=-today.weekday()-1, weeks=2)#sunday 2 
next_sunday = next_sunday.strftime('%Y-%m-%d') #format it
next_monday = today + datetime.timedelta(days=-today.weekday(), weeks=2)#monday2
next_monday = next_monday.strftime('%Y-%m-%d') #format it
following_sunday = today + datetime.timedelta(days=-today.weekday()-1, weeks=3) #sunday 3
following_sunday = following_sunday.strftime('%Y-%m-%d') #format it

mask1 = (df1['CRSD'] <= coming_sunday) #filter for wk1
mask2 = (df1['CRSD'] >= coming_monday) & (df1['CRSD'] <= next_sunday) #filter for wk2
mask3 = (df1['CRSD'] >= next_monday) & (df1['CRSD'] <= following_sunday) #filter for wk3

#define which week the CRSD is in Week 1, 2, or 3
df1['CRSD WK'] = np.where(mask1, 'WK1',
                 np.where(mask2, 'WK2',
                 np.where(mask3, 'WK3','')))

#JUST LEAVE VALUES WHEN MAT TPYE IS SIMPLE
df1['CRSD WK']=np.where(df1['MATL_TYPE']== 'Simple',df1['CRSD WK'],'')

#COLUMN AA, BLUE, COLUMN USUALLY DONE/COPIED AT THE SAME TIME AS RED COLUMN
df1['TPC SKU/DC/WK Key'] = df1["SKU DC Key"] + df1["CRSD WK"]

#lookup Delivery Creation Date
print('4.- Reading the In Play Report')
df2 = pd.read_csv('In Play Report.csv')
df2 = df2[['SO_LINE','Delivery Creation Date']]
df2=df2.drop_duplicates(subset='SO_LINE',keep='first') #df.drop_duplicates as some SO LINES have 2 dates
#first was kept since thats how it was done in excel
df1 = pd.merge(df1, df2, on='SO_LINE', how='left')
df1.rename(columns={'Delivery Creation Date':'SO Line Dropped to DC for Delivery'}, inplace=True)
df1['SO Line Dropped to DC for Delivery']=pd.to_datetime(df1['SO Line Dropped to DC for Delivery'])

#lookup if it is a ZCM Item 
print('5.- Reading ZCM Item Report')
#df3 = pd.read_excel("ZCM Items.xlsx", dtype={'SKU DC': str})
#df3 = df3[['SKU DC','ZCM']]
#df3.rename(columns={'SKU DC': 'SKU DC Key','ZCM':'TPC/ZCM Transmit'},inplace=True)
df3 = pd.read_excel("ZCM Items.xlsx", dtype={'Zebra PN': str,'Zebra Site':str})
df3['SKU DC Key'] = df3['Zebra PN'] + df3['Zebra Site']
df3['TPC/ZCM Transmit'] = "Yes"
df3=df3[['SKU DC Key','TPC/ZCM Transmit']]
df3=df3.drop_duplicates()
df1 = pd.merge(df1, df3, on='SKU DC Key', how='left')
df1['TPC/ZCM Transmit'].fillna('No', inplace = True)

#lookup ASP
print('6.- Reading Fill Rate')
df4 = pd.read_excel("Fill Rate.xlsx", dtype={'Part No': str, 'ASP_Original': float})
df4 = df4[['Part No','ASP_Original']]
df4.rename(columns={'Part No': 'PART','ASP_Original':'Part ASP $'},inplace=True)
df4=df4.drop_duplicates(subset='PART',keep='first')
df1 = pd.merge(df1, df4, on='PART', how='left')
df1['Part ASP $'].fillna(0, inplace=True)
df1.shape

#Item Master printers
print('7.- Reading Item Master printers')
a=fnmatch.filter(os.listdir('.'), 'Item Master - Printers WK*')
a=a[0]    
dfIMP = pd.read_excel(a,header=0,dtype={'SKU DC': str,'Sourcing for':str},usecols="B,L")
dfIMP.rename(columns={'SKU DC': 'SKU DC Key'},inplace=True)
df1 = pd.merge(df1,dfIMP, on='SKU DC Key', how='left')
#df1['Sourcing for'] = np.where(df1['Sourcing for']== 'CTO', df1['Sourcing for'], '')#drop no cto
df1['MATL_TYPE'] = np.where(df1['Sourcing for']== 'CTO', 'Simple Config', df1['MATL_TYPE']) #change cto to simple config 
df1 = df1.drop(['Sourcing for'], axis = 1)
df1.shape

#FINISH CTO Printers    
print('8.- Reading Finish Printer CTO')
a=fnmatch.filter(os.listdir('.'), 'Finish Printer - CTO Agile*')
a=a[0] 
dfCTO = pd.read_excel(a,dtype={'SKU DC': str, 'Item Number (BOM)': str}, sheet_name='CTO')
dfCTO = dfCTO[['SKU DC','Item Number (BOM)']]
dfCTO=dfCTO.drop_duplicates(subset='SKU DC',keep='first')
dfCTO.rename(columns={'SKU DC': 'SKU DC Key'},inplace=True)

df1 = pd.merge(df1,dfCTO, on='SKU DC Key', how='left')
#df1['Part ASP $'].fillna(0, inplace=True)
df1['Config Printer SKU']=df1['Item Number (BOM)']
df1 = df1.drop(['Item Number (BOM)'], axis = 1)

#printers=pd.read_excel('Printers.xlsx')#AUXILIAR WITH YESTERDAYS INFO FROM AIT BL
#df1 = pd.merge(df1,printers, on='SKU DC Key', how='left')
print('9.- Reading ATS from yesterday file')
a=fnmatch.filter(os.listdir('.'), 'ATS Global SP Backlog*')
a=a[0] 
printers = pd.read_excel(a,dtype={'SKU DC Key': str, 'Config Printer SKU': str}, sheet_name='COMBINED',usecols='AA,AB',header=2)
#print(printers.shape,'AIT YESTERDAY')
#printers.head()

printers = printers.dropna(subset = ["Config Printer SKU"]) #JUTS TAKE CONFIG PRINTER ROWS
printers=printers.drop_duplicates() 
#print(printers.shape,'AIT YESTERDAY DISTINCT CONFIGS')

df1 = pd.merge(df1,printers, on='SKU DC Key', how='left')

#df1.loc[df1['SKU DC Key'] == "R4D-0U0A000N-10TX2"]
df1['Config Printer SKU'] = df1['Config Printer SKU_x'].fillna(df1['Config Printer SKU_y'])
#df1.loc[df1['SKU DC Key'] == "R4D-0U0A000N-10TX2"]
df1['Config Printer SKU'] = np.where(df1['MATL_TYPE']== 'Simple Config', df1['Config Printer SKU'],'')
#df1.loc[df1['SKU DC Key'] == "R4D-0U0A000N-10TX2"]

df1 = df1.drop(['Config Printer SKU_x','Config Printer SKU_y'], axis = 1)

#create an auxiliar table where Config Printer SKU is empty, to look up in Agile AIT
dfSourcingRule=df1[df1['Config Printer SKU'].isnull()]
dfSourcingRule=dfSourcingRule[['PART','Config Printer SKU']]
dfSourcingRule=dfSourcingRule.drop_duplicates() 
dfSourcingRule.reset_index(inplace=True) #CREATE NEW INDEXES
dfSourcingRule.drop("index",axis=1,inplace=True)

len(dfSourcingRule) #HOW MANY PARTS WE'LL LOOK FOR IN AGILE EVM
for i in dfSourcingRule.index:
 a=dfSourcingRule['PART'][i]
 print ("Look the next part number in Agile AIT",{a})
 dfSourcingRule['Config Printer SKU'][i] = input()

dfSourcingRule.rename(columns={'Config Printer SKU':'Config Printer SKU1'},inplace=True)
df1 = pd.merge(df1, dfSourcingRule, on='PART', how='left')
df1['Config Printer SKU'] = df1['Config Printer SKU'].fillna(df1['Config Printer SKU1'])

df1 = df1.drop(['Config Printer SKU1'], axis = 1)
df1['Config Printer SKU/DC'] = np.where(df1['MATL_TYPE']== 'Simple Config', df1['Config Printer SKU'] + df1['SHIP_LOC'], '')

df1['Config Printer SKU/DC/WK Key'] = np.where(df1['MATL_TYPE']== 'Simple Config', df1['Config Printer SKU/DC'] + df1['CRSD WK'], '')

#lookup Sourcing Rule, Buyer, Planner Code
print('10.- Reading the Sourcing Rules for EVM')
df5 = pd.read_csv('Sourcing Rule EVM.csv', dtype={'SKU DC': str})
df5 = df5[['SKU DC','Item Organization - Sourcing Rule','Default Buyer','Planner Code']]
df5.rename(columns={'SKU DC': 'SKU DC Key','Item Organization - Sourcing Rule':'Sourcing Rule',
                          'Default Buyer':'BUYER.','Planner Code':'PLANNER CODE'},inplace=True)
df1 = pd.merge(df1, df5, on='SKU DC Key', how='left')
#df1['Sourcing Rule'].fillna(0, inplace=True)#left as empty to replace nulls with other data
df1['BUYER.'].fillna(0, inplace=True)
df1['PLANNER CODE'].fillna(0, inplace=True) 

#FILL NAs from Sourcing Rule WITH AGILE LIST
print('11.- Reading Agile List')
df6 = pd.read_excel("Agile List.xlsx", dtype={'Item Number': str})
df6 = df6[['Item Number','Manufacturer Name']]
df6.rename(columns={'Item Number': 'PART'},inplace =True)
df6=df6.drop_duplicates(subset='PART',keep='first') #some SO LINES have 2 dates, kept first as in excel
df1 = pd.merge(df1, df6, on='PART', how='left')
df1['Sourcing Rule'] = df1['Sourcing Rule'].fillna(df1['Manufacturer Name'])#where sourcing rule is empty
#this values change to the result of the merge, ie the manufacturer name in agile file
df1 = df1.drop(['Manufacturer Name'], axis = 1) #after gettin the values, column from agile is no longer needed
#df1.shape

#retake the sourcing rule file for Config Printer
#df16=pd.read_csv('Sourcing Rule EVM.csv', dtype={'SKU DC': str})
#df16 = df16[['SKU DC','Item Organization - Sourcing Rule']]
df5.rename(columns={'Sourcing Rule':'Sourcing Rule ConfigP','SKU DC Key':'Config Printer SKU/DC'}, inplace = True)
df5=df5[['Sourcing Rule ConfigP','Config Printer SKU/DC']]
df1 = pd.merge(df1, df5, on='Config Printer SKU/DC', how='left')
print(df1.shape)

df1['Sourcing Rule'] = np.where(df1['MATL_TYPE']== 'Simple Config', df1['Sourcing Rule ConfigP'],df1['Sourcing Rule'])

#create XFR column
df1['XFR'] = np.where((df1['Sourcing Rule'].isnull()), '',
             np.where(df1['Sourcing Rule'].str.contains('Transfer'), 'Yes',
             np.where(df1['Sourcing Rule'].str.contains('transfer'), 'Yes', '')))

#if Sourcing Rule starts with Make turn MATL_TYPE to bundle
df1['MATL_TYPEaux'] = np.where(df1['Sourcing Rule'].astype(str).str.startswith('Make'), 'Bundle', df1['MATL_TYPE'])
df1['MATL_TYPE'] = np.where(df1['MATL_TYPE']== 'Simple Config', df1['MATL_TYPE'], df1['MATL_TYPEaux'])
df1 = df1.drop(['MATL_TYPEaux'], axis =1)
#df1 = df1.drop(['Config Printer SKU1','Item Number (BOM)'], axis = 1)

#if MATL_TYPE is Bundle make PART ASP $, UNIT_VAL, TOT_BL all zero
df1['Part ASP $'] = np.where(df1['MATL_TYPE'].astype(str).str.startswith('Bundle'), 0, df1['Part ASP $'])
df1['UNIT_VAL'] = np.where(df1['MATL_TYPE'].astype(str).str.startswith('Bundle'), 0, df1['UNIT_VAL'])
df1['TOT_BL'] = np.where(df1['MATL_TYPE'].astype(str).str.startswith('Bundle'), 0, df1['TOT_BL'])

#create Blended ASP and EXT Part ASP
df1['Blended ASP $'] = np.where((df1['UNIT_VAL'] > 0), df1['UNIT_VAL'], df1['Part ASP $'])
df1['EXT Part ASP $'] = df1['ACTUAL_QTY'] * df1['Blended ASP $']

#AUXILIAR COLUMN, WHEN SIMPLE CONFIG WELL HAVE THE CONFIG SKUDC VALUE, WHEN NOT, WELL HAVE SKU DC VALUE
df1['aux sku dc'] = np.where(df1['MATL_TYPE'] == 'Simple Config', df1['Config Printer SKU/DC'],df1['SKU DC Key'])
#print(df1.shape)

#lookup Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)
print('12.- Reading Inter-Org Intransit')
df7 = pd.read_csv("Inter-Org Intransit.csv", dtype={'ITEM': str})
df7['aux sku dc'] = df7['ITEM'] + df7['ORG'].str[4:]
df7 = pd.pivot_table(df7, values=['DEMAND_SUPPLY_QTY'], index=['aux sku dc'], aggfunc=np.sum)
df1 = pd.merge(df1, df7, on='aux sku dc', how='left')
df1['Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)'] = df1['DEMAND_SUPPLY_QTY']
df1['Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)'].fillna(0, inplace=True)
df1 = df1.drop(['DEMAND_SUPPLY_QTY'], axis = 1)
#print(df1.shape)

#Merge shipment files together
import glob
print("13.- Reading the shipments")
path = r'C:\Users\MJ7255\Desktop\Backlog\INSUMOS ASN' # use your path
all_files = glob.glob(path + "/*.xlsx")

li = []

for filename in all_files:
    df = pd.read_excel(filename, index_col=None, header=1)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
#export ASN file
from datetime import date
#today = date.today()#tday = str(today.strftime("%m/%d/%y")) #tday='ASNs ' + tday #today
#frame.to_excel('ASNs '+str(date.today())+'.xlsx', index=False)
frame=frame[['*Zebra PN','*Ship To Site','*Shipped Quantity']]
frame['aux sku dc']=frame["*Zebra PN"] + frame["*Ship To Site"]
df11 = pd.pivot_table(frame, values=['*Shipped Quantity'], index=['aux sku dc'], aggfunc=np.sum)
df1 = pd.merge(df1, df11, on='aux sku dc', how='left')
df1['ZCM ASN QTY Supply'] = df1['*Shipped Quantity']
df1['ZCM ASN QTY Supply'].fillna(0, inplace=True)
df1 = df1.drop(['*Shipped Quantity'], axis = 1)
#df11.to_excel('ASN PIVOT.xlsx')
#print("ASN file created")
#print(df1.shape)

In [2]:
#lookup weekly ZCM Commits
print('14/14.- Reading All Commit_ZCM')
a=fnmatch.filter(os.listdir('.'), 'All Commit_ZCM*')
a=a[0]

df12 = pd.read_excel(a, header=1, sheet_name='ZCM COMMITS', dtype={'SKU DC': str})
df12 = df12[(df12['Supplier ID'] != 'Supplier ID')] #subset rows with info, there some rows with wronf information, headers in ros and dates in week info
df12 = df12.rename(columns={'SKU DC': 'aux sku dc'})
df12 = df12[['aux sku dc','ALL MISSING','ALL WK 1 & 2','Data Measure','Week 1','Week 2','Week 3','Week 4','Week 5']]

df12 = df12.groupby(['aux sku dc']).sum()
df12.sort_values(by='ALL MISSING', ascending=False, inplace=True)

df1 = pd.merge(df1, df12, on='aux sku dc', how='left')

df1['ZCM Commits WK1'] = df1['Week 1']
df1['ZCM Commits WK1'].fillna(0, inplace=True)
df1['ZCM Commits WK2'] = df1['Week 2']
df1['ZCM Commits WK2'].fillna(0, inplace=True)
df1['ZCM Commits WK3'] = df1['Week 3']
df1['ZCM Commits WK3'].fillna(0, inplace=True)
df1['ZCM Commits WK4'] = df1['Week 4']
df1['ZCM Commits WK4'].fillna(0, inplace=True)
df1['ZCM Commits WK5'] = df1['Week 5']
df1['ZCM Commits WK5'].fillna(0, inplace=True)
df1 = df1.drop(['ALL MISSING','Week 1','Week 2','Week 3','Week 4','Week 5'], axis = 1)

#print(df1.shape, 'numer of rows and columns in COMBINED tab')

nan_value = float("NaN")
dfCB=df1[['SKU DC Key','QOH','Config Printer SKU/DC']]
#print(dfCB.shape,"should be same numer of rows with 3 columns")
dfCB.replace("", nan_value, inplace=True)
dfCB.dropna(subset = ["Config Printer SKU/DC"], inplace=True) #JUTS TAKE CONFIG PRINTER ROWS
#print(dfCB.shape,"number of rows that are Simple Config")
#dfCB.head()

#CREATE SKU DC PIV SHEET
dfCB['QOH_2'] = dfCB['QOH']

dfCBpiv = pd.pivot_table(data=dfCB, index=['SKU DC Key','Config Printer SKU/DC'], values=['QOH','QOH_2'], aggfunc={'QOH': 'sum', 'QOH_2': 'count'}).reset_index()
#generate new column of piv tab

dfCBpiv['Org Part INV-Part Simple Config Total QOH'] = dfCBpiv['QOH'] / dfCBpiv['QOH_2']
#dfCBpiv.head()

dfCBpiv=dfCBpiv[['Config Printer SKU/DC','Org Part INV-Part Simple Config Total QOH']] #just take columns needed
#print(dfCBpiv.shape, "several SKU DC with same config sku dc")
#dfCBpiv = dfCBpiv.drop_duplicates(subset = 'Config Printer SKU/DC',keep='first') done before
#bool_series = pd.notnull(dfCBpiv["Config Printer SKU/DC"])
#dfCBpiv=dfCBpiv[bool_series]
#dfCBpiv=dfCBpiv.drop_duplicates()
#print(dfCBpiv.shape, "Unique Config Printer SKU, first rows kept")
#dfCBpiv.head()

dfCBpiv2 = dfCBpiv.groupby(['Config Printer SKU/DC'],as_index = False).sum()
#print(dfCBpiv2.shape, 'final pivot for  1st column')
#dfCBpiv2.head()

df1 = pd.merge(df1, dfCBpiv2, on='Config Printer SKU/DC', how='left') #first yellow printer column
#print(df1.shape, "Dimensions after merge of 1st piv")
df1['Org Part INV-Part Simple Config Total QOH']=np.where(df1['MATL_TYPE'] == 'Simple Config', df1['Org Part INV-Part Simple Config Total QOH'],0)
df1['Org Part INV-Part Simple Config Total QOH']=pd.to_numeric(df1['Org Part INV-Part Simple Config Total QOH'])

# CREATE CB TO CE COLUMNS IN COMBINED
## TO CREATE A UNIFIED QOH AND THEN CREATE COMBINED BASE+COMBINED IN JUST ONE PIVOT, ASN, INTRANSIT AND COMMITSBY WEEKS 
#ARE ALREADY GENERATED

#READ WORK ORDERS pivot
print('15.-Reading AIT WORK ORDERS')
a=fnmatch.filter(os.listdir('.'), 'AIT WORK ORDERS*')
a=a[0]
dfWO = pd.read_excel(a, header=3, sheet_name='WORK ORDER', dtype={'SKU DC': str}, usecols='N,O')
#MERGE WITH WO PIVOT
dfwopiv1=df1[['SKU DC Key','Config SKU/DC WO Supply Simple Config','Config Printer SKU/DC']]
#print(dfwopiv1.shape, 'data to make WO pivot')

dfwopiv1.replace("", nan_value, inplace=True)
dfwopiv1.dropna(subset = ["Config Printer SKU/DC"], inplace=True)#JUTS TAKE CONFIG PRINTER ROWS
#print(dfwopiv1.shape, 'number of rows that are Simple Config' )

#CREATE SKU DC PIV WO SHEET
dfwopiv = pd.pivot_table(data=dfwopiv1, index=['SKU DC Key'], values=['Config SKU/DC WO Supply Simple Config'], aggfunc={'Config SKU/DC WO Supply Simple Config': 'count'}).reset_index()
#dfwopiv.head()
#dfWO.rename(columns={'SKU DC':'SKU DC Key'}, inplace=True)

dfwopiv = pd.merge(dfwopiv, dfWO, on='SKU DC Key', how='left')
dfwopiv.replace(nan_value, 0,inplace=True)

dfwopiv['NWO'] = dfwopiv['Sum of Quantity'] / dfwopiv['Config SKU/DC WO Supply Simple Config']
#dfwopiv.head()

dfwopiv=dfwopiv[['SKU DC Key','NWO']]
#dfwopiv.head()

#print(df1.shape,'Dimensions after PIV tab, before joining WO tab')

df1 = pd.merge(df1, dfwopiv, on='SKU DC Key', how='left') #second yellow printer column
df1['Config SKU/DC WO Supply Simple Config']=np.where(df1['MATL_TYPE'] == 'Simple Config', df1['NWO'],0)
df1['Config SKU/DC WO Supply Simple Config'] = pd.to_numeric(df1['Config SKU/DC WO Supply Simple Config'])
#dfwopiv.head()
#print(df1.shape,'Dimensions after joining WO tab')

#3drd yellow columnlookup PRINTERS NETTABLE AVILABLE QTY
print('16.Reading Printers Inventory On Hand & Safety Stock')
df22 = pd.read_csv('Printers Inventory On Hand & Safety Stock.csv')
df22 = df22[['SKU DC','Nettable Available Quantity']] #just columns needed
df22.rename(columns={'SKU DC':'Config Printer SKU/DC','Nettable Available Quantity':'Config SKU/DC INV-Simple Config'}, inplace=True)
df1 = pd.merge(df1, df22, on='Config Printer SKU/DC', how='left')
df1['Config SKU/DC INV-Simple Config'] = df1['Config SKU/DC INV-Simple Config'].fillna(0)
#print(df1.shape,'Dimensions after joining Printers Inventory On Hand')

#sum of previous yellow columns, last yellow column
#df1['Org Part INV-Part Simple Config Total QOH']=np.where(df1['MATL_TYPE'] == 'Simple Config', df1['Org Part INV-Part Simple Config Total QOH'],0)
df1['Config SKU/DC INV-Simple Config'] = df1['Config SKU/DC INV-Simple Config'].fillna(0)
df1['Config SKU/DC WO Supply Simple Config'] = pd.to_numeric(df1['Config SKU/DC WO Supply Simple Config'])
df1['Config SKU/DC INV-Simple Config'] = pd.to_numeric(df1['Config SKU/DC INV-Simple Config'])
df1['QOH (Part Simple Config + BASE+ WO Supply)']= df1['Org Part INV-Part Simple Config Total QOH'] + df1['Config SKU/DC WO Supply Simple Config'] + df1['Config SKU/DC INV-Simple Config']
#print(df1.shape,'after sum of yellow columns')

#QOH (Part Simple Config + BASE+ WO Supply)

#AUXILIAR COLUMN, FOR QOH WHEN SIMPLE CONFIG WELL HAVE THE RESULT OF PIVOTS, WHEN SIMPLE, WELL HAVE QOH
#df1['auxQOH'] = np.where(df1['MATL_TYPE'] == 'Simple Config', df1['QOH (Part Simple Config + BASE+ WO Supply)'],df1['QOH'])

#create Short(QTYs On Hand) DBL & CRSD <=+4 columns
#use auxiliar for skudc & qoh
df15 = df1[['SHIP_LOC','Trilogy Or Merlin','CRSD','SO','LINE','SO_LINE','SKU DC Key', 'Config Printer SKU/DC',
            'MATL_TYPE','ACTUAL_QTY','QOH', 'QOH (Part Simple Config + BASE+ WO Supply)']]

#auxiliar of dates to take within 4 days
today = date.today()
four_days = today + datetime.timedelta(days=3)
four_days = four_days.strftime('%Y-%m-%d')
#when is less than five days
df15 = df15[df15['CRSD'] <= four_days]
#df15.head()

#Select SKU Short(QTYs On Hand) DBL & CRSD <=+4 simple
df15simple = df15[df15['MATL_TYPE'] == 'Simple']
df15simple['QOH2'] = df15simple['QOH']
df15simple.reset_index(inplace = True)
df15simple=df15simple.drop(['QOH (Part Simple Config + BASE+ WO Supply)','Config Printer SKU/DC','index'], axis = 1)

#Select SKU Short(QTYs On Hand) DBL & CRSD <=+4 simple config
df15config = df15[df15['MATL_TYPE'] == 'Simple Config']
df15config['QOH SimpleConfig'] = df15config['QOH (Part Simple Config + BASE+ WO Supply)']
df15config.reset_index(inplace = True)
df15config=df15config.drop(['QOH','SKU DC Key','index'],axis = 1)
#df15config.to_excel('DBLCONGIFP.xlsx', index = False)

#create SKU Short(QTYs On Hand) DBL & CRSD <=+4 PIVOT
df16simple = pd.pivot_table(data=df15simple, index=['SKU DC Key'], values=['QOH','QOH2','ACTUAL_QTY'], aggfunc={'QOH': 'sum', 'QOH2': 'count', 'ACTUAL_QTY': 'sum'}).reset_index()
df16simple['SKU/DC QOH'] = df16simple['QOH'] / df16simple['QOH2']
df16simple['Short +/-'] = df16simple['SKU/DC QOH'] - df16simple['ACTUAL_QTY']
df16simple['SKU Short'] = np.where(df16simple['Short +/-'] <= 0, 'YES', 'NO')

#create SKU Short(QTYs On Hand) DBL & CRSD <=+4 PIVOT
df16config = pd.pivot_table(data=df15config, index=['Config Printer SKU/DC'], values=['QOH (Part Simple Config + BASE+ WO Supply)','QOH SimpleConfig','ACTUAL_QTY'], aggfunc={'QOH (Part Simple Config + BASE+ WO Supply)': 'sum', 'QOH SimpleConfig': 'count', 'ACTUAL_QTY': 'sum'}).reset_index()
df16config['SKU/DC QOH'] = df16config['QOH (Part Simple Config + BASE+ WO Supply)'] / df16config['QOH SimpleConfig']
df16config['Short +/-'] = df16config['SKU/DC QOH'] - df16config['ACTUAL_QTY']
df16config['SKU Short'] = np.where(df16config['Short +/-'] <= 0, 'YES', 'NO')

#df16simple.to_excel('1stpivotsimple.xlsx', index = False)
#df16config.to_excel('DBL_CRSD4daysSKUConfigP.xlsx', index = False)
df16simple = df16simple[['SKU DC Key','SKU Short']]
df16config = df16config [['Config Printer SKU/DC','SKU Short']]
#print('1ST PIVOT FOR BOTH SIMPLE AND SIMPLE CONFIG')
#df16.head()

#print(df1.shape,'DIMENSIONS OF main tab before join PIV<4 DIAS for 1st column')
df1 = pd.merge(df1, df16simple, on='SKU DC Key', how='left')
#print(df1.shape,'DIMENSIONS OF main tab after join PIV<4 DIAS for first column')
df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'] = df1['SKU Short']
df1 = df1.drop(['SKU Short'], axis = 1)

df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'] = np.where( (df1['CRSD'] <= four_days) & (df1['MATL_TYPE'] == 'Simple'), df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'], '')
#print('Pivot tab DBL & CRSD <=+4 dimensions version 1, before 2nd column S')

#Create SO Short(QTYs On Hand) DBL & CRSD <=+4 PIVOT
df15simple = pd.merge(df15simple, df16simple, on='SKU DC Key', how='left')
#print('Pivot tab DBL & CRSD <=+4 dimensions final version', df15simple.shape)
#df15.head()

#print(df1.shape,'DIMENSIONS OF main tab before join PIV<4 DIAS for 1st column simple config')
df1 = pd.merge(df1, df16config, on='Config Printer SKU/DC', how='left')
#print(df1.shape,'DIMENSIONS OF main tab after join PIV<4 DIAS for first column')
df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'] = np.where((df1['CRSD'] <= four_days) & (df1['MATL_TYPE'] == 'Simple Config'), 
                                                          df1['SKU Short'], df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'])
df1 = df1.drop(['SKU Short'], axis = 1)
#print('Pivot tab DBL & CRSD <=+4 dimensions version 1, before 2nd column S')

#Create SO Short(QTYs On Hand) DBL & CRSD <=+4 PIVOT
df15config = pd.merge(df15config, df16config, on='Config Printer SKU/DC', how='left')
#print('Pivot tab DBL & CRSD <=+4 dimensions final version config', df15config.shape)
#df15.head()

df17simple = pd.crosstab(index=df15simple["SO"], columns=df15simple["SKU Short"], 
                         values=df15simple["SKU DC Key"], aggfunc='count').reset_index()

df17config = pd.crosstab(index=df15config["SO"], columns=df15config["SKU Short"], 
                         values=df15config["Config Printer SKU/DC"], aggfunc='count').reset_index()
#df17config.head()

#print(df1.shape,'before joining piv simple')
df17simple['SO Short'] = np.where(df17simple['YES'].isnull(), 'NO', 'YES')

df17simple = df17simple[['SO','SO Short']]
df1 = pd.merge(df1, df17simple, on='SO', how='left')
#print(df1.shape,'after joining piv simple')
df17config['SO Shortconfig'] = np.where(df17config['YES'].isnull(), 'NO', 'YES')
df17config = df17config[['SO','SO Shortconfig']]
df1 = pd.merge(df1, df17config, on='SO', how='left')
#print(df1.shape, 'after joining piv simple config')

df1['SO Short(QTYs On Hand) DBL & CRSD <=+4'] = np.where((df1['CRSD'] <= four_days) & (df1['MATL_TYPE'] == 'Simple'), df1['SO Short'], 
                                                np.where((df1['CRSD'] <= four_days) & (df1['MATL_TYPE'] == 'Simple Config'), df1['SO Shortconfig'], ''))

#df1 = df1.drop(['SO Short','SO Shortconfig'], axis = 1)

#print(df1.shape, 'after dropping so shorts and assigning proper ')
df1['DATE'] = pd.to_datetime(df1['DATE'])

#df1['day_of_wk'] = "Wednesday"

#create ZCM ASN/Commit Net WK1 supply
df1['day_of_wk'] = df1['DATE'].dt.day_name()

df1['ZCM ASN/Commit Net WK1 Supply'] = np.where(df1['day_of_wk'] == 'Wednesday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Thursday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Friday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Saturday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Sunday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where((df1['day_of_wk'] == 'Monday') & (df1['ZCM ASN QTY Supply'] >= df1['ZCM Commits WK1']), df1['ZCM ASN QTY Supply'],
                                       np.where((df1['day_of_wk'] == 'Tuesday') & (df1['ZCM ASN QTY Supply'] >= df1['ZCM Commits WK1']), df1['ZCM ASN QTY Supply'], df1['ZCM Commits WK1'])))))))
df1 = df1.drop(['day_of_wk'], axis = 1)

#print(df1.shape)

## 2ND check POINT divide simple vs simple config
#CREATE RED COLUMN for simple
#THE SKU DC WITH DIFERENT COMMENTS WILL KEEP ONLY ONE, IF SEVERAL THE WILL BE CONCATANETED
#Select SKU Short(QTYs On Hand) DBL & CRSD <=+4 simple config
#df15config = df15[df15['MATL_TYPE'] == 'Simple Config']
df13simple=df1[df1['MATL_TYPE'] != 'Simple Config']
df13 = pd.crosstab(index=df13simple["SKU DC Key"], columns=df13simple["CRSD WK"], values=df13simple["CRSD WK"], aggfunc='count').fillna(0)
df13 = df13.reset_index()
df13.sort_values(by='SKU DC Key', ascending=True, inplace=True)

df13['WK1'] = df13['WK1'].astype(int)
df13['WK2'] = df13['WK2'].astype(int)
df13['WK3'] = df13['WK3'].astype(int)
#df13.to_excel('RED COLUMN TABLE1.xlsx', index=False)
df13['WK1'] = np.where(df13['WK1'] > 0, 'Yes', 'No')
df13['WK2'] = np.where(df13['WK2'] > 0, 'Yes', 'No')
df13['WK3'] = np.where(df13['WK3'] > 0, 'Yes', 'No')
df13[''] = np.where(df13[''] > 0, 'Yes', 'No')

#df13 = df13.drop([''], axis = 1)
df13['Missing CRSD Wksimple'] = np.where((df13['WK1'] == 'No') & (df13['WK2'] == 'No') & (df13['WK3'] == 'No'), 'All Missing Blank',
                          np.where((df13['WK1'] == 'No') & (df13['WK2'] == 'Yes') & (df13['WK3'] == 'No') & (df13[''] == 'Yes'), 'Missing WK1,Missing WK3',#
                          np.where((df13['WK1'] == 'No') & (df13['WK2'] == 'No') & (df13['WK3'] == 'Yes'), 'All Missing WK3',
                          np.where((df13[''] == 'Yes') & (df13['WK2'] == 'Yes') & (df13['WK3'] == 'No'), 'Missing WK3',         
                          np.where((df13['WK1'] == 'Yes') & (df13['WK3'] == 'Yes') & (df13['WK2'] == 'No'), 'Missing WK2',
                          np.where((df13['WK2'] == 'Yes') & (df13['WK1'] == 'No'), 'Missing WK1', 'Nothing Missing'))))))
#df13.to_excel('RED COLUMN TABLEsimple.xlsx', index=False)
#print("Red Column for simple created")
df13simple = pd.merge(df13simple, df13, on='SKU DC Key', how='left')
#df1 = df1.drop(['WK1','WK2','WK3'], axis = 1)

df13simple['DISTY']=''
df13simple['SLSOFF2']=''
df13simple['PSION']=''
df13simple['COMPLETE']=''
df13simple['IR_NUMBER']=''
#print('Reordering columns')
df13simple = df13simple[["DATE","THEATRE","SHIP_LOC","Trilogy Or Merlin","ORD_TYPE","SLSOFF2","SLSOFF2_DE","BOOKED","CRSD",
           "Material Avail Date > CRSD","SSD","FSSD","DFF_CRSD","SO","LINE","SO_LINE","SO Line Dropped to DC for Delivery",
           "SKU Short(QTYs On Hand) DBL & CRSD <=+4","SO Short(QTYs On Hand) DBL & CRSD <=+4",
           "SHIP-SET $'s","ORDER $'s","SHIP_SET","PART","Missing CRSD Wksimple","CRSD WK","TPC SKU/DC/WK Key","SKU DC Key",
           "Config Printer SKU","Config Printer SKU/DC","Config Printer SKU/DC/WK Key","TPC/ZCM Transmit","MATL_TYPE",
           "ACTUAL_QTY","REMAIN_QTY","Part ASP $","Blended ASP $",
           "EXT Part ASP $","UNIT_VAL","TOT_BL","TOP-FILE $'s","SOLDTO_NO","SOLD_TO","SHIP_TO","SHIP_DESC","HOLD",
           "DISTY","DISTY_1","TOTAL_ORD","FAMILY","PROD_LINE","CORE","COMPLETE","PO","SOURCE","Sourcing Rule","XFR","SOURCE_2",
           "STATUS","CARRIER","SHIP_MTHD","IR_NUMBER","DAYS_OLD","DAYS_OLD2","BUYER.","PLANNER CODE","BUYER","INVC_QTY",
           "REMAIN_INV","DISTY2","FSSD_CRSD","HOLD_2","REASON","SHIP_QTY","Org Part INV-Part Simple Config Total QOH",
           "Config SKU/DC WO Supply Simple Config","Config SKU/DC INV-Simple Config","QOH (Part Simple Config + BASE+ WO Supply)",
           "QOH","Oracle Intransit(Trilogy IR/ISO)","Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)","ZCM ASN QTY Supply",
           "ZCM Commits WK1","ZCM Commits WK2","ZCM Commits WK3","ZCM Commits WK4","ZCM Commits WK5","WK1 TPC Receipt QTY",
           "Valid TPC Supply","TPC Commits WK1","TPC Commits WK2","TPC Commits WK3","TPC Commits WK4","ZCM ASN/Commit Net WK1 Supply",
           "ZCM ASN QTY and WK1 Net Supply","ZCM Net Commits WK2","ZCM Net Commits WK3","Valid TPC Net Supply WK1",
           "TPC Net Commits WK2","TPC Net Commits WK3","Missing Commits TPC/ZCM WK2","Net Supply capture by CRSD SKU/DC/WK",
           "Missing ASN Supply Due to CRSD WK","Missing Commit Supply Due to CRSD WK","Merlin/Trilogy Missing Supply Total",
           "ORG_BOOK","SKU/DC Total Supply(OH only)","QTY Short to Sales Orders(CUM) Only INV","QTY Short to Sales Orders(NON-CUM) Only INV",
           "CRSD_WEEK","CRSD_MONTH","CRSD_QTR","SSD_WEEK","SSD_MONTH","SSD_QTR","PSION","COUNTRY","DELINQ","ALIGN","BOOK_WEEK",
           "BU","SYSTEM","CREATED_BY","DFF_CRSD2","BOOK_CRSD","BOOK_CRSD2","DAYS_OLD3","SUB_FAMILY","CODE_NAME","THEATRE1",
           "EVM_AIT","HOLD_3","INCO","EVM_AIT2","DELINQ_SSD","DAYS_OLD4","PAY_TERMS","FISC_ZCRD","FISC_SSD","BUSINESS","PRIORITY",
           "STATE","SUB_LINE","EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM) Only INV)","SKU/DC Total Supply(On Hand+ALL ASN)",
           "QTY Short to Sales Orders(CUM) (On Hand+ALL ASN","QTY Short to Sales Orders(NON-CUM) On Hand+ALL ASN",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) On Hand+ALL ASN","SKU/DC Total Supply ALL ASNs+Commits(No Missing Commits)",
           "QTY Short to Sales Orders(CUM) including Missing Commits+Commits+ALL ASNs","QTY Short to Sales Orders(NON-CUM) w/ Commits+Missing Commits+ALL ASNs",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) INV w/ Commits+Missing Commits+ALL ASNs","SKU/DC Total Supply(OH+ALL ASNs+all 3wk Commtsh)",
           "QTY Short to Sales Orders(CUM) OH+ALL ASNs+all 3wk Commts","QTY Short to Sales Orders(NON-CUM) OH+ALL ASNs+all 3wk Commts",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) OH+ALL ASNs+all 3wk Commts"]]

df13simple.sort_values(by=['SKU DC Key','CRSD'],inplace=True)

#DE
#Auxiliar to keep only first rows, it is false in AUX1, AUX1 answers to the question, is this a duplicate row?
df13simple['skuduplicates']=df13simple.duplicated(subset=['SKU DC Key']) #FALSE FOR FIRST APPEARENCE of SKU DC Key,TRUE FOR DUPLICATES
#DUPLICATES FOR SKUDC+WK
df13simple['skuduplicates2']=df13simple.duplicated(subset=['TPC SKU/DC/WK Key'])
#df1 = pd.merge(df1, df13simple, on='Config Printer SKU/DC', how='left')


df13simple['AUX'] = np.where((df13simple['Missing CRSD Wksimple'] == 'Missing WK1,Missing WK3') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']== 'WK2'), 
                            df13simple['ZCM Commits WK1'],
             np.where((df13simple['Missing CRSD Wksimple'] == 'All Missing WK3') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']== 'WK3'),
                      df13simple['ZCM Commits WK1'] + df13simple['ZCM Commits WK2'],
             np.where((df13simple['Missing CRSD Wksimple'] == 'All Missing Blank') & (df13simple['skuduplicates']== False),
                      df13simple['ZCM Commits WK1'] + df13simple['ZCM Commits WK2'] + df13simple['ZCM Commits WK3'],
             np.where((df13simple['Missing CRSD Wksimple'] == 'Missing WK1') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']== 'WK2'),
                      df13simple['ZCM Commits WK1'],
             np.where((df13simple['Missing CRSD Wksimple'] == 'Missing WK2') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']== 'WK3'),
                      df13simple['ZCM Commits WK2'],
             np.where((df13simple['Missing CRSD Wksimple'] == 'Missing WK3') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']== ''),
                      df13simple['ZCM Commits WK3'],
            np.where((df13simple['Missing CRSD Wksimple'] == 'Missing WK1,Missing WK3') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']== ''),
                      df13simple['ZCM Commits WK3'],0)))))))
#COLUMN ED MUTIPLE VLOOKUPS DEPENDING ON RED COLUMN
df13simple['Missing Commit Supply Due to CRSD WK'] = pd.to_numeric(df13simple['AUX'])
df13simple['Missing Commit Supply Due to CRSD WK'].fillna(0, inplace=True)

#ZCM ASN QTY and WK1 Net Supply CV COLUMN
df13simple['ZCM ASN QTY and WK1 Net Supply'] = np.where( (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK'] == 'WK1'),
                                                           df13simple['ZCM ASN/Commit Net WK1 Supply'],0 )

#ZCM Net Commits WK2 CW COLUMN
df13simple['ZCM Net Commits WK2'] = np.where( (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK'] == 'WK2'),
                                                           df13simple['ZCM Commits WK2'],0 )
#ZZCM Net Commits WK3 CX COLUMN
df13simple['ZCM Net Commits WK3'] = np.where( (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK'] == 'WK3'),
                                                           df13simple['ZCM Commits WK3'],0 )

#Net Supply capture by CRSD SKU/DC/WK DC COLUMN  
#CV+CW+CX+CY+CZ+DA
df13simple['ZCM ASN QTY and WK1 Net Supply'] = pd.to_numeric(df13simple['ZCM ASN QTY and WK1 Net Supply'])
df13simple['ZCM Net Commits WK2'] = pd.to_numeric(df13simple['ZCM Net Commits WK2'])
df13simple['ZCM Net Commits WK3'] = pd.to_numeric(df13simple['ZCM Net Commits WK3'])

df13simple['Net Supply capture by CRSD SKU/DC/WK'] = np.where( df13simple['skuduplicates2']== False,
                                                           df13simple['ZCM ASN QTY and WK1 Net Supply'] + df13simple['ZCM Net Commits WK3'] + df13simple['ZCM Net Commits WK2'],0)

df13simple['ZCM ASN QTY Supply'] = pd.to_numeric(df13simple['ZCM ASN QTY Supply'])
#COLUMN DE
df13simple['Missing Commit Supply Due to CRSD WK'] = pd.to_numeric(df13simple['Missing Commit Supply Due to CRSD WK'])

###########NEW
# COLUMN DD =+IF(Z4<>"Nothing Missing",CC4,0)
#bring the ZCM ASN QTY Supply to the Missing ASN Supply Due to CRSD WK column
#Nothing Missing ACCORDING TO RED COLUMN VALUES

df13simple['Missing ASN Supply Due to CRSD WK'] = np.where((df13simple['Missing CRSD Wksimple'] == 'All Missing Blank') & (df13simple['skuduplicates']== False) | (df13simple['Missing CRSD Wksimple'] == 'All Missing WK3') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']=='WK3') 
                                                    | (df13simple['Missing CRSD Wksimple'] == 'Missing WK1,Missing WK3') & (df13simple['skuduplicates2']== False) |
                                                    (df13simple['Missing CRSD Wksimple'] == 'Missing WK1') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']=='WK2') | 
                                                    (df13simple['Missing CRSD Wksimple'] == 'Missing WK2') & (df13simple['skuduplicates2']== False)& (df13simple['CRSD WK']=='WK3') |
                                                    (df13simple['Missing CRSD Wksimple'] == 'Missing WK3') & (df13simple['skuduplicates2']== False) & (df13simple['CRSD WK']==''), 
                                                    df13simple['ZCM ASN QTY Supply'], 0)

df13simple['Missing ASN Supply Due to CRSD WK'] = pd.to_numeric(df13simple['Missing ASN Supply Due to CRSD WK'])



#COLUMN DF Last column of green bucket, copied formulas from yesterday's file
#DF =IF(OR(AA4="WK2",AA4="WK3",AA4=""),CY4,IF(CY4-CX4<0,CX4,CY4))
df13simple['Merlin/Trilogy Missing Supply Total'] = np.where( df13simple['CRSD WK'] != 'WK1',
                                            df13simple['Missing Commit Supply Due to CRSD WK'],
                                            np.where( df13simple['Missing Commit Supply Due to CRSD WK'] - df13simple['Missing ASN Supply Due to CRSD WK'] < 0,
                                            df13simple['Missing ASN Supply Due to CRSD WK'], df13simple['Missing Commit Supply Due to CRSD WK'])) 

df13simple['Merlin/Trilogy Missing Supply Total'] = np.where( (df13simple['Merlin/Trilogy Missing Supply Total'] > 0) & 
                                                      ((df13simple['Missing CRSD Wksimple'] == 'Missing WK2') | (df13simple['Missing CRSD Wksimple'] == 'Missing WK3')),
                                            df13simple['Missing Commit Supply Due to CRSD WK'],df13simple['Merlin/Trilogy Missing Supply Total'])
#print(len(df13simple))

#J COLUMN
df13simple["Material Avail Date > CRSD"]=np.where((df13simple['SSD'] > df13simple['CRSD']), 'Yes', 'No')
df13simple=df13simple.drop(['skuduplicates','skuduplicates2','AUX'], axis=1)

## 2ND check POINT MARISOL config printer
#CREATE RED COLUMN COMBINED BASE
df13simplec=df1[df1['MATL_TYPE'] == 'Simple Config']
df13c = pd.crosstab(index=df13simplec["Config Printer SKU/DC"], columns=df13simplec["CRSD WK"], values=df13simplec["CRSD WK"], 
                    aggfunc='count').fillna(0)
df13c = df13c.reset_index()
df13c.sort_values(by='Config Printer SKU/DC', ascending=True, inplace=True)

df13c['WK1'] = df13c['WK1'].astype(int)
df13c['WK2'] = df13c['WK2'].astype(int)
df13c['WK3'] = df13c['WK3'].astype(int)
#df13.to_excel('RED COLUMN TABLE1.xlsx', index=False)
df13c['WK1'] = np.where(df13c['WK1'] > 0, 'Yes', 'No')
df13c['WK2'] = np.where(df13c['WK2'] > 0, 'Yes', 'No')
df13c['WK3'] = np.where(df13c['WK3'] > 0, 'Yes', 'No')
df13c[''] = np.where(df13c[''] > 0, 'Yes', 'No')

#df13 = df13.drop([''], axis = 1)
df13c['Missing CRSD Wksimple'] = np.where((df13c['WK1'] == 'No') & (df13c['WK2'] == 'No') & (df13c['WK3'] == 'No'), 'All Missing Blank',
                          np.where((df13c['WK1'] == 'No') & (df13c['WK2'] == 'Yes') & (df13c['WK3'] == 'No') & (df13c[''] == 'Yes'), 'Missing WK1,Missing WK3',#
                          np.where((df13c['WK1'] == 'No') & (df13c['WK2'] == 'No') & (df13c['WK3'] == 'Yes'), 'All Missing WK3',
                          np.where((df13c[''] == 'Yes') & (df13c['WK2'] == 'Yes') & (df13c['WK3'] == 'No'), 'Missing WK3',         
                          np.where((df13c['WK1'] == 'Yes') & (df13c['WK3'] == 'Yes') & (df13c['WK2'] == 'No'), 'Missing WK2',
                          np.where((df13c['WK2'] == 'Yes') & (df13c['WK1'] == 'No'), 'Missing WK1', 'Nothing Missing'))))))

#df13c.to_excel('RED COLUMN TABLEconfig.xlsx', index=False)
#print("Red Column for simple config created")
df13simplec = pd.merge(df13simplec, df13c, on='Config Printer SKU/DC', how='left')

#df1 = df1.drop(['WK1','WK2','WK3'], axis = 1)
df13simplec['DISTY']=''
df13simplec['SLSOFF2']=''
df13simplec['PSION']=''
df13simplec['COMPLETE']=''
df13simplec['IR_NUMBER']=''
#print('Reordering columns')
df13simplec = df13simplec[["DATE","THEATRE","SHIP_LOC","Trilogy Or Merlin","ORD_TYPE","SLSOFF2","SLSOFF2_DE","BOOKED","CRSD",
           "Material Avail Date > CRSD","SSD","FSSD","DFF_CRSD","SO","LINE","SO_LINE","SO Line Dropped to DC for Delivery",
           "SKU Short(QTYs On Hand) DBL & CRSD <=+4","SO Short(QTYs On Hand) DBL & CRSD <=+4",
           "SHIP-SET $'s","ORDER $'s","SHIP_SET","PART","Missing CRSD Wksimple","CRSD WK","TPC SKU/DC/WK Key","SKU DC Key",
           "Config Printer SKU","Config Printer SKU/DC","Config Printer SKU/DC/WK Key","TPC/ZCM Transmit","MATL_TYPE",
           "ACTUAL_QTY","REMAIN_QTY","Part ASP $","Blended ASP $",
           "EXT Part ASP $","UNIT_VAL","TOT_BL","TOP-FILE $'s","SOLDTO_NO","SOLD_TO","SHIP_TO","SHIP_DESC","HOLD",
           "DISTY","DISTY_1","TOTAL_ORD","FAMILY","PROD_LINE","CORE","COMPLETE","PO","SOURCE","Sourcing Rule","XFR","SOURCE_2",
           "STATUS","CARRIER","SHIP_MTHD","IR_NUMBER","DAYS_OLD","DAYS_OLD2","BUYER.","PLANNER CODE","BUYER","INVC_QTY",
           "REMAIN_INV","DISTY2","FSSD_CRSD","HOLD_2","REASON","SHIP_QTY","Org Part INV-Part Simple Config Total QOH",
           "Config SKU/DC WO Supply Simple Config","Config SKU/DC INV-Simple Config","QOH (Part Simple Config + BASE+ WO Supply)",
           "QOH","Oracle Intransit(Trilogy IR/ISO)","Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)","ZCM ASN QTY Supply",
           "ZCM Commits WK1","ZCM Commits WK2","ZCM Commits WK3","ZCM Commits WK4","ZCM Commits WK5","WK1 TPC Receipt QTY",
           "Valid TPC Supply","TPC Commits WK1","TPC Commits WK2","TPC Commits WK3","TPC Commits WK4","ZCM ASN/Commit Net WK1 Supply",
           "ZCM ASN QTY and WK1 Net Supply","ZCM Net Commits WK2","ZCM Net Commits WK3","Valid TPC Net Supply WK1",
           "TPC Net Commits WK2","TPC Net Commits WK3","Missing Commits TPC/ZCM WK2","Net Supply capture by CRSD SKU/DC/WK",
           "Missing ASN Supply Due to CRSD WK","Missing Commit Supply Due to CRSD WK","Merlin/Trilogy Missing Supply Total",
           "ORG_BOOK","SKU/DC Total Supply(OH only)","QTY Short to Sales Orders(CUM) Only INV","QTY Short to Sales Orders(NON-CUM) Only INV",
           "CRSD_WEEK","CRSD_MONTH","CRSD_QTR","SSD_WEEK","SSD_MONTH","SSD_QTR","PSION","COUNTRY","DELINQ","ALIGN","BOOK_WEEK",
           "BU","SYSTEM","CREATED_BY","DFF_CRSD2","BOOK_CRSD","BOOK_CRSD2","DAYS_OLD3","SUB_FAMILY","CODE_NAME","THEATRE1",
           "EVM_AIT","HOLD_3","INCO","EVM_AIT2","DELINQ_SSD","DAYS_OLD4","PAY_TERMS","FISC_ZCRD","FISC_SSD","BUSINESS","PRIORITY",
           "STATE","SUB_LINE","EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM) Only INV)","SKU/DC Total Supply(On Hand+ALL ASN)",
           "QTY Short to Sales Orders(CUM) (On Hand+ALL ASN","QTY Short to Sales Orders(NON-CUM) On Hand+ALL ASN",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) On Hand+ALL ASN","SKU/DC Total Supply ALL ASNs+Commits(No Missing Commits)",
           "QTY Short to Sales Orders(CUM) including Missing Commits+Commits+ALL ASNs","QTY Short to Sales Orders(NON-CUM) w/ Commits+Missing Commits+ALL ASNs",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) INV w/ Commits+Missing Commits+ALL ASNs","SKU/DC Total Supply(OH+ALL ASNs+all 3wk Commtsh)",
           "QTY Short to Sales Orders(CUM) OH+ALL ASNs+all 3wk Commts","QTY Short to Sales Orders(NON-CUM) OH+ALL ASNs+all 3wk Commts",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) OH+ALL ASNs+all 3wk Commts"]]

#print('Done Reordering of columns')
df13simplec.sort_values(by=['SKU DC Key','CRSD'],inplace=True)

#DE
#Auxiliar to keep only first rows, it is false in AUX1, AUX1 answers to the question, is this a duplicate row?
df13simplec['skuduplicates']=df13simplec.duplicated(subset=['Config Printer SKU/DC']) #FALSE FOR ALL FIRST APPEARENCE of SKU DC Key,TRUE FOR DUPLICATES
#DUPLICATES FOR SKUDC+WK
df13simplec['skuduplicates2']=df13simplec.duplicated(subset=['Config Printer SKU/DC/WK Key'])
#df1 = pd.merge(df1, df13simple, on='Config Printer SKU/DC', how='left')


df13simplec['AUX'] = np.where((df13simplec['Missing CRSD Wksimple'] == 'Missing WK1,Missing WK3') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']== 'WK2'), 
                            df13simplec['ZCM Commits WK1'],
             np.where((df13simplec['Missing CRSD Wksimple'] == 'All Missing WK3') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']== 'WK3'),
                      df13simplec['ZCM Commits WK1'] + df13simplec['ZCM Commits WK2'],
             np.where((df13simplec['Missing CRSD Wksimple'] == 'All Missing Blank') & (df13simplec['skuduplicates']== False),
                      df13simplec['ZCM Commits WK1'] + df13simplec['ZCM Commits WK2'] + df13simplec['ZCM Commits WK3'],
             np.where((df13simplec['Missing CRSD Wksimple'] == 'Missing WK1') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']== 'WK2'),
                      df13simplec['ZCM Commits WK1'],
             np.where((df13simplec['Missing CRSD Wksimple'] == 'Missing WK2') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']== 'WK3'),
                      df13simplec['ZCM Commits WK2'],
             np.where((df13simplec['Missing CRSD Wksimple'] == 'Missing WK3') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']== ''),
                      df13simplec['ZCM Commits WK3'],
            np.where((df13simplec['Missing CRSD Wksimple'] == 'Missing WK1,Missing WK3') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']== ''),
                      df13simplec['ZCM Commits WK3'],0)))))))
#COLUMN ED MUTIPLE VLOOKUPS DEPENDING ON RED COLUMN
df13simplec['Missing Commit Supply Due to CRSD WK'] = pd.to_numeric(df13simplec['AUX'])
df13simplec['Missing Commit Supply Due to CRSD WK'].fillna(0, inplace=True)

#ZCM ASN QTY and WK1 Net Supply CQ COLUMN
df13simplec['ZCM ASN QTY and WK1 Net Supply'] = np.where( (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK'] == 'WK1'),
                                                           df13simplec['ZCM ASN/Commit Net WK1 Supply'],0 )

#ZCM Net Commits WK2 CR COLUMN
df13simplec['ZCM Net Commits WK2'] = np.where( (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK'] == 'WK2'),
                                                           df13simplec['ZCM Commits WK2'],0 )
#ZZCM Net Commits WK3 CS COLUMN
df13simplec['ZCM Net Commits WK3'] = np.where( (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK'] == 'WK3'),
                                                           df13simplec['ZCM Commits WK3'],0 )

#Net Supply capture by CRSD SKU/DC/WK CX COLUMN  
#CV+CW+CX+CY+CZ+DA
df13simplec['ZCM ASN QTY and WK1 Net Supply'] = pd.to_numeric(df13simplec['ZCM ASN QTY and WK1 Net Supply'])
df13simplec['ZCM Net Commits WK2'] = pd.to_numeric(df13simplec['ZCM Net Commits WK2'])
df13simplec['ZCM Net Commits WK3'] = pd.to_numeric(df13simplec['ZCM Net Commits WK3'])

df13simplec['Net Supply capture by CRSD SKU/DC/WK'] = np.where( df13simplec['skuduplicates2']== False, df13simplec['ZCM ASN QTY and WK1 Net Supply'] + df13simplec['ZCM Net Commits WK3'] + df13simplec['ZCM Net Commits WK2'],0)

df13simplec['ZCM ASN QTY Supply'] = pd.to_numeric(df13simplec['ZCM ASN QTY Supply'])
#COLUMN DE
df13simplec['Missing Commit Supply Due to CRSD WK'] = pd.to_numeric(df13simplec['Missing Commit Supply Due to CRSD WK'])

###########NEW
# COLUMN DD =+IF(Z4<>"Nothing Missing",CC4,0)
#bring the ZCM ASN QTY Supply to the Missing ASN Supply Due to CRSD WK column
#Nothing Missing ACCORDING TO RED COLUMN VALUES

df13simplec['Missing ASN Supply Due to CRSD WK'] = np.where((df13simplec['Missing CRSD Wksimple'] == 'All Missing Blank') & (df13simplec['skuduplicates']== False) | (df13simplec['Missing CRSD Wksimple'] == 'All Missing WK3') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']=='WK3') 
                                                    | (df13simplec['Missing CRSD Wksimple'] == 'Missing WK1,Missing WK3') & (df13simplec['skuduplicates2']== False) |
                                                    (df13simplec['Missing CRSD Wksimple'] == 'Missing WK1') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']=='WK2') | 
                                                    (df13simplec['Missing CRSD Wksimple'] == 'Missing WK2') & (df13simplec['skuduplicates2']== False)& (df13simplec['CRSD WK']=='WK3') |
                                                    (df13simplec['Missing CRSD Wksimple'] == 'Missing WK3') & (df13simplec['skuduplicates2']== False) & (df13simplec['CRSD WK']==''), 
                                                    df13simplec['ZCM ASN QTY Supply'], 0)

df13simplec['Missing ASN Supply Due to CRSD WK'] = pd.to_numeric(df13simplec['Missing ASN Supply Due to CRSD WK'])



#COLUMN DF Last column of green bucket, copied formulas from yesterday's file
#DF =IF(OR(AA4="WK2",AA4="WK3",AA4=""),CY4,IF(CY4-CX4<0,CX4,CY4))
df13simplec['Merlin/Trilogy Missing Supply Total'] = np.where( df13simplec['CRSD WK'] != 'WK1',
                                            df13simplec['Missing Commit Supply Due to CRSD WK'],
                                            np.where( df13simplec['Missing Commit Supply Due to CRSD WK'] - df13simplec['Missing ASN Supply Due to CRSD WK'] < 0,
                                            df13simplec['Missing ASN Supply Due to CRSD WK'], df13simplec['Missing Commit Supply Due to CRSD WK'])) 

df13simplec['Merlin/Trilogy Missing Supply Total'] = np.where( (df13simplec['Merlin/Trilogy Missing Supply Total'] > 0) & 
                                                      ((df13simplec['Missing CRSD Wksimple'] == 'Missing WK2') | (df13simplec['Missing CRSD Wksimple'] == 'Missing WK3')),
                                            df13simplec['Missing Commit Supply Due to CRSD WK'],df13simplec['Merlin/Trilogy Missing Supply Total'])

#J COLUMN
df13simplec["Material Avail Date > CRSD"]=np.where((df13simplec['SSD'] > df13simplec['CRSD']), 'Yes', 'No')
df13simplec=df13simplec.drop(['skuduplicates','skuduplicates2','AUX'], axis=1)

df13simplec.sort_values(by=['Config Printer SKU/DC','CRSD'],inplace=True)
print('Writing file')
strings = time.strftime("%m.%d.%Y")
strings= 'ATS ' + strings 
b=("%s.xlsx" % strings)

with pd.ExcelWriter(b) as writer:  
    #dfASN.to_excel(writer, sheet_name='CRM',index= False)
    df13simple.to_excel(writer, sheet_name='COMBINED',index= False,startrow=2)
    df13simplec.to_excel(writer, sheet_name='COMBINED BASE',index= False,startrow=2)
    
c=(time.time() - start_time)/60
print("--- %s minutes to create AIT BL---" % c)

14/14.- Reading All Commit_ZCM


C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-2-9f769ad25727>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCB.dropna(subset = ["Config Printer SKU/DC"], inplace=True) #JUTS TAKE CONFIG PRINTER ROWS
<ipython-input-2-9f769ad25727>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

15.-Reading AIT WORK ORDERS


C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-2-9f769ad25727>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfwopiv1.dropna(subset = ["Config Printer SKU/DC"], inplace=True)#JUTS TAKE CONFIG PRINTER ROWS


16.Reading Printers Inventory On Hand & Safety Stock


<ipython-input-2-9f769ad25727>:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df15simple['QOH2'] = df15simple['QOH']
<ipython-input-2-9f769ad25727>:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df15config['QOH SimpleConfig'] = df15config['QOH (Part Simple Config + BASE+ WO Supply)']


Writing file
--- 34.77385207017263 minutes to create AIT BL---


In [3]:
a=fnmatch.filter(os.listdir('.'), 'All Commit_ZCM*')
a=a[0]
#print(a)
#print(b)
pathtc = "C:\\Users\\MJ7255\\Desktop\\Backlog\\Backlog Report SAM\\Backlog Report\\Untitled Folder\\" + a
pathf = "C:\\Users\\MJ7255\\Desktop\\Backlog\\Backlog Report SAM\\Backlog Report\\Untitled Folder\\" + b
wbtc = xw.Book(pathtc)
wbf = xw.Book(pathf)
wstc = wbtc.sheets(1)

wstc.api.Copy(After=wbf.sheets(2).api)
wbf.save()
wbtc.app.quit()

In [1]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import xlwings as xw
from datetime import date
df1 = pd.DataFrame({'a':[1,2,3,4],'b':[1,2,3,4],'c':[1,2,3,4]})
df2 = pd.DataFrame({'a':[1,2,3,4],'c':[1,2,3,4]})

df = pd.concat([df1,df2])

print('df1')
print(df1)
print('\ndf2')
print(df2)
print('\ndf')
print(df)

df1
   a  b  c
0  1  1  1
1  2  2  2
2  3  3  3
3  4  4  4

df2
   a  c
0  1  1
1  2  2
2  3  3
3  4  4

df
   a    b  c
0  1  1.0  1
1  2  2.0  2
2  3  3.0  3
3  4  4.0  4
0  1  NaN  1
1  2  NaN  2
2  3  NaN  3
3  4  NaN  4
